In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# this dataset if for badminton review
review1 = pd.read_csv("data.csv")
review1.head()

,Reviewer Name,Review Title,Place of Review,Up Votes,Down Votes,Month,Review text,Ratings
0,Kamal Suresh,Nice product,"Certified Buyer, Chirakkal",889.0,64.0,Feb 2021,"Nice product, good quality, but price is now r...",4
1,Flipkart Customer,Don't waste your money,"Certified Buyer, Hyderabad",109.0,6.0,Feb 2021,They didn't supplied Yonex Mavis 350. Outside ...,1
2,A. S. Raja Srinivasan,Did not meet expectations,"Certified Buyer, Dharmapuri",42.0,3.0,Apr 2021,Worst product. Damaged shuttlecocks packed in ...,1
3,Suresh Narayanasamy,Fair,"Certified Buyer, Chennai",25.0,1.0,NaN,"Quite O. K. , but nowadays the quality of the...",3
4,ASHIK P A,Over priced,NaN,147.0,24.0,Apr 2016,Over pricedJust â?¹620 ..from retailer.I didn'...,1


In [3]:
review1.drop_duplicates(inplace=True)

In [4]:
review1.isna().sum()

Reviewer Name        6
Review Title         6
Place of Review     46
Up Votes             6
Down Votes           6
Month              461
Review text          4
Ratings              0
dtype: int64

In [5]:
review1.dropna(inplace=True)

In [6]:
review1.isna().sum()

Reviewer Name      0
Review Title       0
Place of Review    0
Up Votes           0
Down Votes         0
Month              0
Review text        0
Ratings            0
dtype: int64

In [7]:
review1.shape

(8013, 8)

In [8]:
def assign_ratings(ratings):
    if ratings == 5 or ratings == 4:
        return "Positive"
    else:
        return "Negative"


review1["Target"]=review1["Ratings"].apply(assign_ratings)

In [9]:
review1.Target.value_counts()

Target
Positive    6462
Negative    1551
Name: count, dtype: int64

In [10]:
cleaned_df = review1[['Review text', 'Target']]

In [11]:
# cleaned_df.duplicated().sum()

In [12]:
# cleaned_df.drop_duplicates(inplace=True)

In [13]:
cleaned_df.duplicated().sum()

3203

In [14]:
cleaned_df.isna().sum()

Review text    0
Target         0
dtype: int64

In [15]:
cleaned_df.shape

(8013, 2)

In [16]:
cleaned_df.head()

,Review text,Target
0,"Nice product, good quality, but price is now r...",Positive
1,They didn't supplied Yonex Mavis 350. Outside ...,Negative
2,Worst product. Damaged shuttlecocks packed in ...,Negative
5,Good quality product. Delivered on time.READ MORE,Positive
6,BEST PURCHASE It is a good quality and is more...,Positive


In [17]:
cleaned_df['target']=cleaned_df['Target'].map({'Positive':1,'Negative':0})

C:\Users\Manisha\AppData\Local\Temp\ipykernel_10516\2359280761.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_df['target']=cleaned_df['Target'].map({'Positive':1,'Negative':0})


In [18]:
x=cleaned_df['Review text']
y=cleaned_df['target']

In [19]:
y.value_counts()

target
1    6462
0    1551
Name: count, dtype: int64

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=42, stratify=y)

In [21]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Parameters
max_words = 5000
max_len = 50

# Tokenizer
tokenizer = Tokenizer(num_words=max_words, oov_token='<oov>')
tokenizer.fit_on_texts(x_train)

# Convert text to sequences
x_train_seq = tokenizer.texts_to_sequences(x_train)
x_test_seq = tokenizer.texts_to_sequences(x_test)

# Pad sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=max_len, padding='post')
x_test_pad = pad_sequences(x_test_seq, maxlen=max_len, padding='post')


In [22]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Bidirectional, LSTM, Dense, GRU
from tensorflow.keras.layers import SpatialDropout1D, Dropout, GRU

In [23]:
model = Sequential([
    Embedding(input_dim=max_words, output_dim=100, input_length=max_len),
    SpatialDropout1D(0.2),
    GRU(128, dropout=0.2, recurrent_dropout=0.2),
    Dense(32, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model.build(input_shape=(None, max_len))
model.summary()


C:\Users\Manisha\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 50, 100)             │         500,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ spatial_dropout1d (SpatialDropout1D) │ (None, 50, 100)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ (None, 128)                 │          88,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 32)                  │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 592,481 (2.26 MB)

 Trainable params: 592,481 (2.26 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)


In [25]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

es = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mc = ModelCheckpoint('best_gru_model.h5', monitor='val_loss', save_best_only=True)

history = model.fit(
    x_train_pad, y_train,
    validation_data=(x_test_pad, y_test),
    epochs=10,
    batch_size=32,
    callbacks=[es, mc],
    verbose=1
)


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.7829 - loss: 0.5250

201/201 ━━━━━━━━━━━━━━━━━━━━ 32s 116ms/step - accuracy: 0.7830 - loss: 0.5249 - val_accuracy: 0.8066 - val_loss: 0.4915
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 105ms/step - accuracy: 0.8068 - loss: 0.4997 - val_accuracy: 0.8066 - val_loss: 0.4935
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.8121 - loss: 0.4899

201/201 ━━━━━━━━━━━━━━━━━━━━ 22s 107ms/step - accuracy: 0.8121 - loss: 0.4899 - val_accuracy: 0.8066 - val_loss: 0.4876
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step - accuracy: 0.8412 - loss: 0.4099

201/201 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - accuracy: 0.8413 - loss: 0.4095 - val_accuracy: 0.9027 - val_loss: 0.3009
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - accuracy: 0.9235 - loss: 0.2436

201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 106ms/step - accuracy: 0.9235 - loss: 0.2436 - val_accuracy: 0.9064 - val_loss: 0.2732
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 21s 105ms/step - accuracy: 0.9382 - loss: 0.2114 - val_accuracy: 0.8977 - val_loss: 0.2873
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 22s 108ms/step - accuracy: 0.9458 - loss: 0.1915 - val_accuracy: 0.8977 - val_loss: 0.3030
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 22s 109ms/step - accuracy: 0.9528 - loss: 0.1717 - val_accuracy: 0.8983 - val_loss: 0.3117


In [26]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred_probs = model.predict(x_test_pad)
y_pred = (y_pred_probs >= 0.5).astype(int)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


51/51 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step
Accuracy: 0.9064254522769807

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.65      0.73       310
           1       0.92      0.97      0.94      1293

    accuracy                           0.91      1603
   macro avg       0.87      0.81      0.84      1603
weighted avg       0.90      0.91      0.90      1603


Confusion Matrix:
 [[ 203  107]
 [  43 1250]]


In [27]:
def predict_sentiment(text, model, tokenizer, max_len=50):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    prob = model.predict(padded)[0][0]
    label = "Positive" if prob >= 0.5 else "Negative"
    confidence = prob if prob >= 0.5 else 1 - prob
    return label, round(confidence, 2)

# Example
review = "The quality is terrible and it stopped working after just two days."
label, conf = predict_sentiment(review, model, tokenizer, max_len)
print(f"{label} (confidence: {conf})")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step
Negative (confidence: 0.85)


In [29]:
# Example
review = "This product is amazing and totally worth the price"
label, conf = predict_sentiment(review, model, tokenizer, max_len)
print(f"{label} (confidence: {conf})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step
Positive (confidence: 0.9700000286102295)
